In [1]:
!conda env list

# conda environments:
#
base                  *  /opt/conda
federated_learning       /opt/conda/envs/federated_learning
python2                  /opt/conda/envs/python2
pytorch1.8               /opt/conda/envs/pytorch1.8
rapids                   /opt/conda/envs/rapids
tf1.14                   /opt/conda/envs/tf1.14
tf2.4.1                  /opt/conda/envs/tf2.4.1



In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModelForSequenceClassification
from tqdm import tqdm
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, classification_report
import wandb

/root/.local/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-05-08 00:30:18.526064: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib64
2022-05-08 00:30:18.526437: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib64
2022-05-08 00:30:18.526453: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing l

In [3]:
!nvidia-smi
wandb.init(project="senti-analysis", entity='cocoshe', mode='disabled')
# wandb.config = {
#     'batch_size': 32,
#     'epochs': 5,
#     'lr': 5e-5,
#     'max_seq_length': 128,
# }
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Sun May  8 00:30:24 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 450.51.05    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  Off  | 00000000:3E:00.0 Off |                    0 |
| N/A   35C    P0    39W / 250W |      0MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


In [4]:
class MyDataset(Dataset):
    def __init__(self, df, model_name='bert-base-chinese'):
        self.data_df = df
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

    def __len__(self):
        return len(self.data_df)

    def __getitem__(self, idx):
        input_text = self.data_df.iloc[idx, 0]
        label = self.data_df.iloc[idx, 1]

        tokenizer = self.tokenizer
        token = tokenizer(input_text, padding='max_length', truncation=True, max_length=128, return_tensors='pt', return_token_type_ids=False)
        input_ids = token['input_ids'].squeeze(0)
        attention_mask = token['attention_mask'].squeeze(0)
        label = np.int64(label)
        return input_ids.to(device), attention_mask.to(device), label

In [5]:
tokenizer = AutoTokenizer.from_pretrained('hfl/chinese-roberta-wwm-ext')
model = AutoModelForSequenceClassification.from_pretrained('hfl/chinese-roberta-wwm-ext', num_labels=5)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
epochs = 20

Some weights of the model checkpoint at hfl/chinese-roberta-wwm-ext were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model che

In [6]:
def train_test_split(input_df, test_size=0.2, shuffle=True, random_state=None):
    if shuffle:
        input_df = input_df.sample(frac=1)
    ret_train_df = input_df[:int(len(input_df) * (1 - test_size))]
    ret_test_df = input_df[int(len(input_df) * (1 - test_size)):]
    return ret_train_df, ret_test_df


In [7]:
evaluate2score = {'力荐': '4', '推荐': '3', '还行': '2', '较差': '1', '很差': '0'}
score2evaluate = {4: '力荐', '3': '推荐', '2': '还行', '1': '较差', '0': '很差'}
evaluate_list = ['力荐', '推荐', '还行', '较差', '很差']
k, v = list(evaluate2score.items())[0]

In [8]:
# split train and test data directly(no need to split input and label, that's why not use sklearn.model_selection.train_test_split)
df = pd.read_csv('dataset/流浪地球.csv', encoding='utf-8', header=0, index_col=None, usecols=['content', 'evaluate'])
print(df.drop_duplicates(['evaluate'])['evaluate'])
df = df[df['evaluate'].isin(evaluate_list)]  # filter out the evaluate not in evaluate_list
print('after filter')
print(df.drop_duplicates(['evaluate'])['evaluate'])
df.head()

0                       推荐
2                       很差
3                       力荐
5                       还行
6                       较差
56     2019-02-05 23:16:40
96     2019-02-08 11:53:57
119    2019-02-09 09:14:52
161    2019-01-28 20:14:00
247    2019-02-10 23:30:50
261    2019-02-06 20:33:03
307    2019-02-06 15:31:25
339    2019-02-10 18:09:43
354    2019-02-08 05:09:17
395    2019-02-08 00:15:25
417    2019-02-09 18:32:26
431    2019-02-08 22:39:41
446    2019-02-07 19:41:14
463    2019-02-08 22:00:00
Name: evaluate, dtype: object
after filter
0    推荐
2    很差
3    力荐
5    还行
6    较差
Name: evaluate, dtype: object


,content,evaluate
0,一个悲伤的故事：太阳都要毁灭，地球都要流浪了，我国的校服还是这么丑......,推荐
1,电影比预期要更恢弘磅礴，晨昏线过后的永夜、火种计划、让地球流浪、木星推动地球…等等大小设定，...,推荐
2,还能更土更儿戏一点吗？毫无思考仅靠煽动，毫无敬畏仅余妄想。好的科幻片应该首先承认人类的无知，...,很差
3,1.终于，轮到我们仰望星空。2.后启示录死亡废墟，赛博朋克地下城，以及烟波浩渺的末日想象，缔...,力荐
4,真为吴京的演技尴尬，总是摆出一副大义凌然的样子，好奇为什么刘的作品中总有这种傻逼般的圣母存在...,很差


In [9]:
df['evaluate'] = df['evaluate'].map(evaluate2score).astype(int)
print(df.dtypes)
df.head()


content     object
evaluate     int64
dtype: object


,content,evaluate
0,一个悲伤的故事：太阳都要毁灭，地球都要流浪了，我国的校服还是这么丑......,3
1,电影比预期要更恢弘磅礴，晨昏线过后的永夜、火种计划、让地球流浪、木星推动地球…等等大小设定，...,3
2,还能更土更儿戏一点吗？毫无思考仅靠煽动，毫无敬畏仅余妄想。好的科幻片应该首先承认人类的无知，...,0
3,1.终于，轮到我们仰望星空。2.后启示录死亡废墟，赛博朋克地下城，以及烟波浩渺的末日想象，缔...,4
4,真为吴京的演技尴尬，总是摆出一副大义凌然的样子，好奇为什么刘的作品中总有这种傻逼般的圣母存在...,0


In [10]:
train_df, test_df = train_test_split(df, test_size=0.3, shuffle=True)

train_dataset = MyDataset(train_df, model_name='hfl/chinese-roberta-wwm-ext')
test_dataset = MyDataset(test_df, model_name='hfl/chinese-roberta-wwm-ext')

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)


In [11]:
# train
def train():
    model.train()
    for epoch in range(epochs):
        for i, batch in enumerate(tqdm(train_loader)):
            input_ids, attention_mask, label = batch
            input_ids, attention_mask, label = input_ids.to(device), attention_mask.to(device), label.to(device)
            print(label)
            output = model(input_ids, attention_mask, labels=label)
            loss = output[0]

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            print('epoch: ', epoch, 'loss: ', loss.item())
            wandb.log({'epoch': epoch,'step': i, 'loss': loss.item()})

        evaluate()

In [12]:
# evaluate
def evaluate():
    label_list = []
    pred_list = []
    model.eval()
    with torch.no_grad():
        model.eval()
        eval_steps = 0
        pred_list = []
        label_list = []
        for i, batch in enumerate(tqdm(test_loader)):
            input_ids, attention_mask, label = batch
            input_ids, attention_mask, label = input_ids.to(device), attention_mask.to(device), label.to(device)
            output = model(input_ids, attention_mask, labels=label)
            pred_list += (torch.argmax(output[1], dim=-1))
            label_list += label
            eval_steps += 1

        label_list, pred_list = [item.cpu().numpy() for item in label_list], [item.cpu().numpy() for item in pred_list]
        # f1 = f1_score(label_list, pred_list, average='macro')
        # acc = accuracy_score(label_list, pred_list)
        # precision = precision_score(label_list, pred_list, average='macro')
        # recall = recall_score(label_list, pred_list, average='macro')
        report = classification_report(label_list, pred_list)
        print(report)

In [ ]:
train()
# evaluate()
# torch.save(model.state_dict(), 'model/bert_cls_model.pth')

  0%|          | 0/11 [00:00<?, ?it/s]

tensor([4, 0, 0, 3, 1, 4, 2, 3, 4, 0, 2, 4, 4, 0, 1, 4, 0, 3, 4, 2, 1, 3, 4, 1,
        2, 3, 1, 3, 4, 2, 4, 2], device='cuda:0')


  9%|▉         | 1/11 [00:01<00:12,  1.23s/it]

epoch:  0 loss:  1.6250149011611938
tensor([2, 3, 2, 2, 3, 1, 2, 4, 4, 2, 2, 2, 1, 2, 4, 3, 3, 1, 3, 4, 3, 2, 2, 2,
        1, 2, 4, 2, 1, 1, 3, 2], device='cuda:0')


 18%|█▊        | 2/11 [00:01<00:07,  1.28it/s]

...

 36%|███▋      | 4/11 [00:03<00:05,  1.26it/s]

epoch:  16 loss:  0.005192482843995094
tensor([4, 4, 1, 0, 2, 4, 3, 3, 2, 3, 3, 4, 2, 1, 0, 0, 3, 2, 4, 4, 2, 3, 1, 2,
        2, 4, 3, 4, 2, 2, 3, 2], device='cuda:0')


 45%|████▌     | 5/11 [00:03<00:04,  1.28it/s]

epoch:  16 loss:  0.004657974001020193
tensor([2, 3, 2, 1, 1, 4, 2, 3, 3, 2, 4, 1, 0, 1, 4, 0, 0, 1, 3, 2, 2, 1, 3, 0,
        4, 4, 2, 0, 3, 3, 1, 3], device='cuda:0')


 55%|█████▍    | 6/11 [00:04<00:04,  1.24it/s]